In [57]:
import sys
import os
current_dir = os.path.dirname(os.path.abspath('./'))
if not current_dir in sys.path:
    sys.path.append(current_dir)
from combiners import EpochsCombiner
from typing import *
import mne
import tensorflow as tf
import mneflow as mf
import matplotlib.pyplot as plt
import numpy as np
from utils.data_management import dict2str
from lfcnn_vis import plot_patterns
from LFCNN_decoder import SpatialParameters, TemporalParameters, ComponentsOrder, Predictions
import pickle
from utils.machine_learning import one_hot_decoder
import sklearn.metrics as sm
from utils.machine_learning.confusion import ConfusionEstimator

def read_pkl(path: str) -> Any:
    with open(
            path,
            'rb'
        ) as file:
        content = pickle.load(
            file
        )
    return content

In [100]:
import pandas as pd

class PredictionsParser(object):
    def __init__(
        self,
        y_true: Union[list[int], np.ndarray],
        y_pred: Union[list[int], np.ndarray],
        class_names: Optional[Union[str, list[str]]] = None
    ):
        
        y_true = self.__check_numpy(y_true)
        y_pred = self.__check_numpy(y_pred)
        
        if y_true.shape != y_pred.shape:
            raise ValueError('Predictions and actual values are inconsistent. Actual values shape: {y_true.shape}, predictions shape: {y_pred.shape}')
        
        if len(y_true.shape) != 1:
            y_true = one_hot_decoder(y_true)
            y_pred = one_hot_decoder(y_pred)
        
        self._y_true = y_true
        self._y_pred = y_pred
        
        self._accuracy = sm.accuracy_score(y_true, y_pred)
        
        classes_true = np.unique(self._y_true)
        classes_pred = np.unique(self._y_pred)
        
        if np.any(classes_true != classes_pred):
            raise ValueError(f'Classes are inconsistent.\n\tActual classes: {classes_true}\n\tPredicted classes: {classes_pred}')
        
        del classes_pred
        
        self._classes = classes_true
        self._n_classes = len(classes_true)
        
        if class_names is not None:
            
            if isinstance(class_names, str):
                class_names = class_names.split(' ')
            
            if len(class_names) != self.n_classes:
                raise ValueError(f'Class names and classes are inconsistent: number of classes is {self.n_classes}, but {len(class_names)} names of classes were given')
        else:
            class_names = [f'Class {i}' for i in range(self.n_classes)]
        
        self._class_names = class_names
        
        self._confusion = pd.DataFrame(
            sm.confusion_matrix(self.y_true, self.y_pred),
            index = [f'Actual {class_name}' for class_name in self.class_names],
            columns = [f'Predicted {class_name}' for class_name in self.class_names]
        )
    
    @staticmethod
    def __check_numpy(arr: Union[list, tuple, np.ndarray]):
        if isinstance(arr, np.ndarray):
            return arr
        elif not isinstance(arr, np.ndarray) and isinstance(arr, (list, tuple)):
            return np.array(arr)
        else:
            raise ValueError(f'The given argument must be either a np.ndarray or a list, but {type(arr)} was given')
        
    @property
    def y_true(self):
        return self._y_true
    @y_true.setter
    def y_true(self, value):
        raise AttributeError('Impossible to set y_true directly')
    
    @property
    def y_pred(self):
        return self._y_pred
    @y_pred.setter
    def y_pred(self, value):
        raise AttributeError('Impossible to set y_pred directly')
    
    @property
    def accuracy(self):
        return self._accuracy
    @accuracy.setter
    def accuracy(self, value):
        raise AttributeError('Impossible to set accuracy directly')
    
    @property
    def classes(self):
        return self._classes
    @classes.setter
    def classes(self, value):
        raise AttributeError('Impossible to set classes directly')
    
    @property
    def n_classes(self):
        return self._n_classes
    @n_classes.setter
    def n_classes(self, value):
        raise AttributeError('Impossible to set number of classes directly')
    
    @property
    def class_names(self):
        return self._class_names
    @class_names.setter
    def class_names(self, value):
        raise AttributeError('Impossible to set names for classes directly')
    
    @property
    def confusion(self):
        return self._confusion
    @confusion.setter
    def confusion(self, value):
        raise AttributeError('Impossible to set confusion matrix directly')
    
    def summary(self):
        df = self.confusion.copy()
        summary = pd.DataFrame(columns = self.class_names)
        summary.loc['Total'] = [df[column].sum() for column in df.columns]
        summary.loc['Accuracy'] = [None, None, None, None]
        summary.loc['Accuracy2'] = [None, None, None, None]
        ec = self.estimate_confusion()
        for i, (column, class_name) in enumerate(zip(df.columns, summary.columns)):
            summary[class_name].loc['Accuracy'] = df[column].iloc[i]/summary[class_name]['Total']
            summary[class_name].loc['Accuracy2'] = ec[class_name].acc
        return summary
    
    def estimate_confusion(self):
        

        for i, class_name in enumerate(self.class_names):
            
            tp = self.confusion[self.confusion.columns[i]][self.confusion.index[i]]
            fn = self.confusion[
                list(self.confusion.columns[:i]) + list(self.confusion.columns[i+1:])
                ].loc[self.confusion.index[i]].sum()
            fp = self.confusion[self.confusion.columns[i]].loc[
                list(self.confusion.index[:i]) + list(self.confusion.index[i+1:])
            ].sum()
            tn = self.confusion[
                list(self.confusion.columns[:i]) + list(self.confusion.columns[i+1:])
                ].loc[
                    list(self.confusion.index[:i]) + list(self.confusion.index[i+1:])
                ].sum().sum()
            print(class_name, tp, tn, fn, fp, (tp+tn)/(tp+tn+fp+fn))
        print(self.confusion)
        return {
            class_name: ConfusionEstimator(
                # tp, tn, fp, fn
                self.confusion[self.confusion.columns[i]][self.confusion.index[i]],
                self.confusion[
                    list(self.confusion.columns[:i]) + list(self.confusion.columns[i+1:])
                    ].loc[
                        list(self.confusion.index[:i]) + list(self.confusion.index[i+1:])
                ].sum().sum(),
                self.confusion[self.confusion.columns[i]].loc[
                    list(self.confusion.index[:i]) + list(self.confusion.index[i+1:])
                ].sum(),
                self.confusion[
                    list(self.confusion.columns[:i]) + list(self.confusion.columns[i+1:])
                ].loc[self.confusion.index[i]].sum()
            )
            for i, class_name in enumerate(self.class_names)
        }
            # break



In [101]:


path = '../Source/Subjects/Ga_Fed_06'

predictions = read_pkl(
    os.path.join(path, 'Predictions', 'LM_vs_LI_vs_RM_vs_RI_B1-B8_pred.pkl')
)

pp = PredictionsParser(predictions.y_true, predictions.y_p, 'LM LI RM RI')
print(pp.accuracy)
pp.summary()

0.5471698113207547
LM 10 31 6 6 0.7735849056603774
LI 8 33 8 4 0.7735849056603774
RM 8 33 4 8 0.7735849056603774
RI 3 38 6 6 0.7735849056603774
           Predicted LM  Predicted LI  Predicted RM  Predicted RI
Actual LM            10             4             1             1
Actual LI             6             8             1             1
Actual RM             0             0             8             4
Actual RI             0             0             6             3


,LM,LI,RM,RI
Total,16,12,16,9
Accuracy,0.625,0.666667,0.5,0.333333
Accuracy2,0.773585,0.773585,0.773585,0.773585


In [93]:
path = '../Source/Subjects/Ga_Fed_06'
predictions = read_pkl(
    os.path.join(path, 'Predictions', 'LM&LI_vs_RM&RI_B1-B8_pred.pkl')
)
y_t = one_hot_decoder(predictions.y_true)
y_p = one_hot_decoder(predictions.y_p)
print(sm.confusion_matrix(y_t, y_p))
print(len(y_p))
# tn, fp, fn, tp = sm.confusion_matrix(y_t, y_p).ravel()
# ce = BinaryConfusionEstimator(tp, tn, fp, fn)
# for t, p in zip(y_t, y_p):
#     print(t, p)

[[25  1]
 [ 0 25]]
51


In [4]:


subjects_path = '../Source/Subjects/'

for subject_name in os.listdir(subjects_path):
    
    if subject_name == 'Pse_Udo':
        continue
    
    subject_path = os.path.join(subjects_path, subject_name)

    predictions = read_pkl(
        os.path.join(subject_path, 'Predictions', 'LM&LI_vs_RM&RI_B1-B8_pred.pkl')
    )

    y_t = one_hot_decoder(predictions.y_true)

    y_p = one_hot_decoder(predictions.y_p)

    print(subject_name)
    tn, fp, fn, tp = sm.confusion_matrix(y_t, y_p).ravel()
    ce = ConfusionEstimator(tp, tn, fp, fn)

    print(
        f'\tT\tF\n'\
        f'P\t{tp}\t{fp}\n'\
        f'N\t{tn}\t{fn}\n'
        f'Accuracy: {ce.acc}\n'\
        f'Specificity: {ce.spec}\n'\
        f'Sensitivity: {ce.sens}\n'
    )

Ga_Fed_06
	T	F
P	25	1
N	25	0
Accuracy: 0.9803921568627451
Specificity: 0.9615384615384616
Sensitivity: 1.0

Fe_To_08
	T	F
P	29	0
N	24	2
Accuracy: 0.9636363636363636
Specificity: 1.0
Sensitivity: 0.9354838709677419

Az_Mar_05
	T	F
P	23	2
N	28	0
Accuracy: 0.9622641509433962
Specificity: 0.9333333333333333
Sensitivity: 1.0

Ku_EL_09
	T	F
P	25	2
N	24	1
Accuracy: 0.9423076923076923
Specificity: 0.9230769230769231
Sensitivity: 0.9615384615384616

Pr_An_04
	T	F
P	27	6
N	10	8
Accuracy: 0.7254901960784313
Specificity: 0.625
Sensitivity: 0.7714285714285715

Ru_Ek_07
	T	F
P	22	0
N	29	1
Accuracy: 0.9807692307692307
Specificity: 1.0
Sensitivity: 0.9565217391304348

Se_Tu_03
	T	F
P	24	1
N	31	1
Accuracy: 0.9649122807017544
Specificity: 0.96875
Sensitivity: 0.96

Te_Ali_11
	T	F
P	24	4
N	20	5
Accuracy: 0.8301886792452831
Specificity: 0.8333333333333334
Sensitivity: 0.8275862068965517

